In [2]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image

# -------------------- Config --------------------
train_dir = "/home/intellisense08/Yehan/project_dir/Webots/Root_data/train/"
num_classes = 3
batch_size = 16
epochs = 30
learning_rate = 1e-4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------- Dataset --------------------
class MultiLabelDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []
        for subfolder in os.listdir(root_dir):
            sub_path = os.path.join(root_dir, subfolder)
            if not os.path.isdir(sub_path):
                continue
            # folder name encodes label e.g. "1_0_1"
            label_vector = [int(x) for x in subfolder.split('_')]
            for fname in os.listdir(sub_path):
                if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                    self.images.append(os.path.join(sub_path, fname))
                    self.labels.append(label_vector)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float)

        img = Image.open(img_path).convert("L")  # grayscale

        if self.transform:
            img = self.transform(img)

        # expand grayscale to 3 channels
        img = img.repeat(3, 1, 1)

        return img, label


# -------------------- Transforms --------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

train_dataset = MultiLabelDataset(train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

# -------------------- ResNet50 (from scratch) --------------------
model = models.resnet50(pretrained=False)  # no pretrained weights
model.fc = nn.Linear(model.fc.in_features, num_classes)  # replace final layer
model = model.to(device)

# -------------------- Loss & Optimizer --------------------
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# -------------------- Accuracy Function --------------------
def exact_match_accuracy(outputs, targets, threshold=0.5):
    preds = (torch.sigmoid(outputs) >= threshold).int()
    targets = targets.int()
    correct = (preds == targets).all(dim=1).sum().item()
    total = targets.size(0)
    return correct, total

# -------------------- Training Loop --------------------
for epoch in range(epochs):
    model.train()
    running_loss, running_correct, running_total = 0.0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        correct, total = exact_match_accuracy(outputs, labels)
        running_correct += correct
        running_total += total

    train_loss = running_loss / running_total
    train_acc = running_correct / running_total

    print(f"Epoch [{epoch+1}/{epochs}] "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")

# -------------------- Save Model --------------------
save_path = "/home/intellisense08/Yehan/project_dir/Webots/trained_models/Resnet50/resnet50_synthetic.pth"
save_path_old = "/home/intellisense08/Yehan/project_dir/Webots/trained_models/Resnet50/resnet50_synthetic_old.pth"
torch.save(model.state_dict(), save_path)
torch.save(model.state_dict(), save_path_old,_use_new_zipfile_serialization=False)
print(f"Model saved to {save_path}")


Epoch [1/30] Train Loss: 0.3460, Train Acc: 0.6246
Epoch [2/30] Train Loss: 0.1212, Train Acc: 0.8742
Epoch [3/30] Train Loss: 0.0642, Train Acc: 0.9395
Epoch [4/30] Train Loss: 0.0505, Train Acc: 0.9488
Epoch [5/30] Train Loss: 0.0294, Train Acc: 0.9734
Epoch [6/30] Train Loss: 0.0292, Train Acc: 0.9758
Epoch [7/30] Train Loss: 0.0231, Train Acc: 0.9781
Epoch [8/30] Train Loss: 0.0165, Train Acc: 0.9859
Epoch [9/30] Train Loss: 0.0131, Train Acc: 0.9895
Epoch [10/30] Train Loss: 0.0178, Train Acc: 0.9836
Epoch [11/30] Train Loss: 0.0264, Train Acc: 0.9742
Epoch [12/30] Train Loss: 0.0115, Train Acc: 0.9887
Epoch [13/30] Train Loss: 0.0038, Train Acc: 0.9984
Epoch [14/30] Train Loss: 0.0081, Train Acc: 0.9934
Epoch [15/30] Train Loss: 0.0074, Train Acc: 0.9945
Epoch [16/30] Train Loss: 0.0136, Train Acc: 0.9859
Epoch [17/30] Train Loss: 0.0039, Train Acc: 0.9977
Epoch [18/30] Train Loss: 0.0087, Train Acc: 0.9945
Epoch [19/30] Train Loss: 0.0029, Train Acc: 0.9977
Epoch [20/30] Train L